## پروژه اول درس هوش مصنوعی با موضوع الگوریتم های جستجوی آگاهانه و ناآگاهانه

### در این پروژه سعی میکنیم با استفاده از الگوریتم های زیر مساله ی یافتن راهی برای جمع آوری معجون ها از معبدی با شکل شطرنجی را حل کنیم

## BFS , IDS & A*

# Imports

In [1]:
#import numpy as np
import time
from copy import deepcopy
import queue as qu

# Body

## Load tests data

In [2]:
tests = []
file = open("test1.in")
tests.append(file.readlines())
file.close()
file = open("test2.in")
tests.append(file.readlines())
file.close()
file = open("test3.in")
tests.append(file.readlines())
file.close()

#### داده های تست ها را از فایل هایی که در پوشه پروژه قرار دارند برای تست برنامه لود می کنیم
#### با توجه به تست های موجود فرض می شود که تعداد سطر ها و ستون ها اعداد تک رقمی می باشند 

## Create tests maps

In [3]:
maps = []
Cs = []
Ks = []
for test in tests:
    num_of_row = int(test[0][0])
    num_of_col = int(test[0][2])
    curr_map = [['e'] * num_of_col for i in range(num_of_row)]
    curr_c = int(test[1][0])
    curr_k = int(test[1][2])
    curr_Cs = []
    curr_Ks = []
    for i in range(0, curr_c):
        curr_Cs.append([int(test[2+i][0]), int(test[2+i][2]), False])
        curr_map[int(test[2+i][0])][int(test[2+i][2])] = 'c'
    for i in range(0, curr_k):
        curr_Ks.append([int(test[2+curr_c+i][0]), int(test[2+curr_c+i][2]), False])
        curr_map[int(test[2+curr_c+i][0])][int(test[2+curr_c+i][2])] = 'k'
    curr_d = int(test[2+curr_c+curr_k])
    for i in range(0, curr_d):
        curr_map[int(test[3+curr_c+curr_k+i][0])][int(test[3+curr_c+curr_k+i][2])] = 'd'
    maps.append(curr_map)
    Cs.append(curr_Cs)
    Ks.append(curr_Ks)


In [4]:
maps

[[['e', 'k', 'c', 'e'],
  ['e', 'e', 'd', 'e'],
  ['c', 'e', 'e', 'e'],
  ['e', 'e', 'e', 'e']],
 [['k', 'e', 'e', 'd'],
  ['e', 'k', 'c', 'd'],
  ['c', 'c', 'e', 'e'],
  ['e', 'k', 'e', 'e']],
 [['e', 'e', 'd', 'e', 'e', 'e'],
  ['e', 'd', 'c', 'e', 'e', 'c'],
  ['e', 'e', 'k', 'e', 'e', 'e'],
  ['d', 'e', 'e', 'e', 'e', 'c'],
  ['e', 'd', 'k', 'c', 'e', 'e'],
  ['e', 'c', 'e', 'e', 'e', 'e']]]

#### مشاهده می شود که نقشه های تست به درستی  لود شده اند

## Model Description

### Actions : Di up, Di down, Di right, Di left (for i in range num of doctors)

### Initial State : A doctor in (0, 0) and the map looks like the test files

### Goal State : Every doctor in (n-1, m-1) and no 'c' remain in the map

### Path cost : sum of moves (Actions) of all doctors (from Initial state to Goal state)

### State : position of elements in the map in a node represents the current state for this node

### Transition Model : اگر دکتری به خانه ای که دارای دارو است وارد شود آن خانه تبدیل به خانه ی خالی شده و یک دکتر در خانه ی پایین سمت چپ جدول ایجاد می شود . اگر دکتری وارد خانه ای دارای معجون شود آن خانه به خانه ی خالی تبدیل می شود . برای حرکت دکتر ها هم تنها در صورتی که خانه ی مورد نظر مجاز باشد و قابل رفتن باشد به آنجا می رود

## Common Functions

In [5]:
def is_in_list(node, ls, bfs):
    #result = np.where(expanded == curr_state, True, False)
    #if True in result:
    #    return True
    #return False
    ##############
    if bfs:
        for nd in ls:
            if nd.is_same_for_BFS(node):
                return True
        return False
    else:
        for nd in ls:
            if nd.is_same_for_IDS(node):
                return True
        return False
    ############
    #node_list = [node]*len(expanded)
    #result = map(is_same, node_list, expanded)
    #if True in result:
    #    return True
    #return False
    
def eat_drug(Ks, x, y):
    for k in Ks:
        if k[0] == x and k[1] == y:
            if k[2]:
                return False
            else:
                k[2] = True
                return True
            
def pick_confection(Cs, x, y):
    for c in Cs:
        if c[0] == x and c[1] == y:
            c[2] = True
            return
        
def is_dr_in_list(dr, drs, m, n):
    if dr[1] == m-1 and dr[2] == n-1:
        return True
    for d in drs:
        if d[1] == dr[1] and d[2] == dr[2]:
            return True
    return False

## Data Structures

In [6]:
class Fifo:
    def __init__(self):
        self.ls = []
        self.front = 0
        self.rear = 0
        
    def put(self, element):
        self.ls.append(element)
        self.rear += 1
        
    def get(self):
        self.front += 1
        return self.ls[self.front-1]
    
    def empty(self):
        return self.front >= self.rear
        

In [7]:
class Lifo:
    def __init__(self):
        self.ls = []
        self.top = 0
        
    def put(self, element):
        self.ls.append(element)
        self.top += 1
        
    def get(self):
        self.top -= 1
        return self.ls.pop()
    
    def empty(self):
        return self.top <= 0
        

In [8]:
class Node:
    def __init__(self, curr_map, curr_Cs, curr_Ks, path_until_there, path_cost_until_there, curr_Drs, row, col):
        self.map = curr_map
        self.Cs = curr_Cs
        self.Ks = curr_Ks
        self.path = path_until_there
        self.path_cost = path_cost_until_there
        self.Drs = curr_Drs
        self.m = row
        self.n = col
        #self.state = self.calculate_state()
        if self.path_cost == 0:
            if self.map[0][0] == 'k':
                eat_drug(self.Ks, 0, 0)
                self.Drs.append([1, self.m-1, 0])
                
    #def calculate_state(self):
        #string = ""
        #for c in self.Cs:
            #if c[2]:
                #string += '1'
            #else:
                #string += '0'
        #for k in self.Ks:
            #if k[2]:
                #string += '1'
            #else:
                #string += '0'
        #dr_ls = []
        #for dr in self.Drs:
            #dr_ls.append(10 * dr[1] + dr[2])
        #dr_ls.sort()
        #string += str(dr_ls)
        #return string
    
    def Goal_test(self):
        for c in self.Cs:
            if c[2] is False:
                return False
        for dr in self.Drs:
            if dr[1] != self.m-1 or dr[2] != self.n-1:
                return False
        return True
    
    def expand(self, queue, expanded):
        expanded.append(self)
        for dr in self.Drs:
            if dr[1] == self.m-1 and dr[2] == self.n-1:
                continue
            if dr[1]+1 < self.m:
                if self.map[dr[1]+1][dr[2]] != 'd':
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][1] = dr[1]+1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]+1][dr[2]] == 'k':
                        if eat_drug(temp_Ks, dr[1]+1, dr[2]):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]+1][dr[2]] == 'c':
                        pick_confection(temp_Cs, dr[1]+1, dr[2])
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "D"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded, True):
                        if temp_node.Goal_test():
                            return temp_node
                        if not is_in_list(temp_node, queue.ls, True):
                            queue.put(temp_node)
            if dr[1] > 0:
                if self.map[dr[1]-1][dr[2]] != 'd':
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][1] = dr[1]-1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]-1][dr[2]] == 'k':
                        if eat_drug(temp_Ks, dr[1]-1, dr[2]):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]-1][dr[2]] == 'c':
                        pick_confection(temp_Cs, dr[1]-1, dr[2])
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "U"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded, True):
                        if temp_node.Goal_test():
                            return temp_node
                        if not is_in_list(temp_node, queue.ls, True):
                            queue.put(temp_node)
            if dr[2]+1 < self.n:
                if self.map[dr[1]][dr[2]+1] != 'd':
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][2] = dr[2]+1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]][dr[2]+1] == 'k':
                        if eat_drug(temp_Ks, dr[1], dr[2]+1):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]][dr[2]+1] == 'c':
                        pick_confection(temp_Cs, dr[1], dr[2]+1)
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "R"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded, True):
                        if temp_node.Goal_test():
                            return temp_node
                        if not is_in_list(temp_node, queue.ls, True):
                            queue.put(temp_node)
            if dr[2] > 0:
                if self.map[dr[1]][dr[2]-1] != 'd':
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][2] = dr[2]-1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]][dr[2]-1] == 'k':
                        if eat_drug(temp_Ks, dr[1], dr[2]-1):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]][dr[2]-1] == 'c':
                        pick_confection(temp_Cs, dr[1], dr[2]-1)
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "L"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded, True):
                        if temp_node.Goal_test():
                            return temp_node
                        if not is_in_list(temp_node, queue.ls, True):
                            queue.put(temp_node)
            break  
        return None
    
    def RDLS(self, expanded, limit):
        if self.Goal_test():
            return self
        if limit == 0:
            return None
        expanded.append(self)
        for dr in self.Drs:
            if dr[1] == self.m-1 and dr[2] == self.n-1:
                continue
            if dr[1]+1 < self.m:
                if self.map[dr[1]+1][dr[2]] != 'd':
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][1] = dr[1]+1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]+1][dr[2]] == 'k':
                        if eat_drug(temp_Ks, dr[1]+1, dr[2]):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]+1][dr[2]] == 'c':
                        pick_confection(temp_Cs, dr[1]+1, dr[2])
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "D"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded, False):
                        result = temp_node.RDLS(expanded, limit-1)
                        if result != None:
                            return result
            if dr[1] > 0:
                if self.map[dr[1]-1][dr[2]] != 'd':
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][1] = dr[1]-1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]-1][dr[2]] == 'k':
                        if eat_drug(temp_Ks, dr[1]-1, dr[2]):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]-1][dr[2]] == 'c':
                        pick_confection(temp_Cs, dr[1]-1, dr[2])
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "U"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded, False):
                        result = temp_node.RDLS(expanded, limit-1)
                        if result != None:
                            return result
            if dr[2]+1 < self.n:
                if self.map[dr[1]][dr[2]+1] != 'd':
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][2] = dr[2]+1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]][dr[2]+1] == 'k':
                        if eat_drug(temp_Ks, dr[1], dr[2]+1):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]][dr[2]+1] == 'c':
                        pick_confection(temp_Cs, dr[1], dr[2]+1)
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "R"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded, False):
                        result = temp_node.RDLS(expanded, limit-1)
                        if result != None:
                            return result
            if dr[2] > 0:
                if self.map[dr[1]][dr[2]-1] != 'd':
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][2] = dr[2]-1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]][dr[2]-1] == 'k':
                        if eat_drug(temp_Ks, dr[1], dr[2]-1):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]][dr[2]-1] == 'c':
                        pick_confection(temp_Cs, dr[1], dr[2]-1)
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "L"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded, False):
                        result = temp_node.RDLS(expanded, limit-1)
                        if result != None:
                            return result
            break
        return None
        
    def is_same_for_BFS(self, node):
        if self.Cs == node.Cs and self.Ks == node.Ks:
            for dr in self.Drs:
                if not is_dr_in_list(dr, node.Drs, self.m, self.n):
                    return False
            return True
        return False
    
    def is_same_for_IDS(self, node):
        if len(self.Drs) == len(node.Drs) and self.path_cost == node.path_cost:
            if self.Cs == node.Cs and self.Ks == node.Ks:
                if self.Drs == node.Drs:
                    return True
        return False

## Test Implementation with BFS

### این الگوریتم کامل است و همواره جواب بهینه را به دست می آورد ولی از نظر تعداد استیت های دیده شده و زمان اجرا بهینه نیست 

## My Test

In [9]:
maps_copy = deepcopy(maps)

In [10]:
Cs_copy = deepcopy(Cs)

In [11]:
maps_copy[0]

[['e', 'k', 'c', 'e'],
 ['e', 'e', 'd', 'e'],
 ['c', 'e', 'e', 'e'],
 ['e', 'e', 'e', 'e']]

In [12]:
Cs_copy[0] = [[0, 2, False]]

In [13]:
maps_copy[0][2][0] = 'e'

In [14]:
maps_copy[0]

[['e', 'k', 'c', 'e'],
 ['e', 'e', 'd', 'e'],
 ['e', 'e', 'e', 'e'],
 ['e', 'e', 'e', 'e']]

In [15]:
fifo = Fifo()
expanded = []

In [16]:
first_node = Node(maps_copy[0], Cs_copy[0], Ks[0], [], 0, [[0, 0, 0]], len(maps_copy[0]), len(maps_copy[0][0]))
fifo.put(first_node)
#frontier.append(first_node)
#index = 0

In [17]:
first_time = time.time()
while not fifo.empty():
    curr_node = fifo.get()
    answer = curr_node.expand(fifo, expanded)
    if answer != None:
        print("answer found")
        print("path cost =", answer.path_cost)
        print(answer.path)
        break
last_time = time.time()

answer found
path cost = 9
[[0, 'R'], [0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D'], [1, 'R'], [1, 'R'], [1, 'R']]


In [18]:
bfs_time = last_time - first_time

In [19]:
bfs_time

0.002584695816040039

In [20]:
len(expanded)

36

## Test 1

In [21]:
fifo = Fifo()
expanded = []

In [22]:
fifo.put(Node(maps[0], Cs[0], Ks[0], [], 0, [[0, 0, 0]], len(maps[0]), len(maps[0][0])))

In [23]:
first_time = time.time()
while not fifo.empty():
    curr_node = fifo.get()
    answer = curr_node.expand(fifo, expanded)
    if answer != None:
        print("answer found")
        print("path cost =", answer.path_cost)
        print(answer.path)
        break
last_time = time.time()

answer found
path cost = 11
[[0, 'R'], [0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D'], [1, 'U'], [1, 'R'], [1, 'D'], [1, 'R'], [1, 'R']]


In [24]:
bfs_time = last_time - first_time

In [25]:
bfs_time

0.005033254623413086

In [26]:
len(expanded)

63

## Test 2

In [27]:
fifo = Fifo()
expanded = []

In [28]:
fifo.put(Node(maps[1], Cs[1], Ks[1], [], 0, [[0, 0, 0]], len(maps[1]), len(maps[1][0])))

In [29]:
first_time = time.time()
while not fifo.empty():
    curr_node = fifo.get()
    answer = curr_node.expand(fifo, expanded)
    if answer != None:
        print("answer found")
        print("path cost =", answer.path_cost)
        print(answer.path)
        break
last_time = time.time()

answer found
path cost = 11
[[0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D'], [0, 'R'], [1, 'U'], [1, 'R'], [1, 'R'], [1, 'D'], [1, 'R']]


In [30]:
bfs_time = last_time - first_time

In [31]:
bfs_time

0.02100372314453125

In [32]:
len(expanded)

157

## Test 3

In [33]:
fifo = Fifo()
expanded = []

In [34]:
fifo.put(Node(maps[2], Cs[2], Ks[2], [], 0, [[0, 0, 0]], len(maps[2]), len(maps[2][0])))

In [35]:
first_time = time.time()
while not fifo.empty():
    curr_node = fifo.get()
    answer = curr_node.expand(fifo, expanded)
    if answer != None:
        print("answer found")
        print("path cost =", answer.path_cost)
        print(answer.path)
        break
last_time = time.time()

answer found
path cost = 19
[[0, 'D'], [0, 'D'], [0, 'R'], [0, 'R'], [0, 'U'], [0, 'R'], [0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D'], [0, 'D'], [1, 'R'], [1, 'R'], [1, 'R'], [1, 'U'], [1, 'D'], [1, 'R'], [1, 'R']]


In [36]:
bfs_time = last_time - first_time

In [37]:
bfs_time

0.48108839988708496

In [38]:
len(expanded)

1010

## Test Implementation with IDS

### این الگوریتم کامل است و همواره جواب بهینه را می دهد ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

### زمان اجرای این الگوریتم از بی اف اس نیز بیشتر است چرا که به ازای هر بار که لیمیت افزایش پیدا می کند تمام نود ها با ارتفاع کمتر که در مرحله ی قبل دیده شده اند باز بررسی می شوند

## Test 1

In [39]:
first_time = time.time() 
answer = None
all_expanded = []
limit = 1

while answer == None:
    all_expanded.extend(expanded)
    expanded = []
    first_node = Node(maps[0], Cs[0], Ks[0], [], 0, [[0, 0, 0]], len(maps[0]), len(maps[0][0]))
    answer = first_node.RDLS(expanded, limit)
    limit += 1
last_time = time.time()

print("answer found")
print("path cost =", answer.path_cost)
print(answer.path)

answer found
path cost = 11
[[0, 'R'], [0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D'], [1, 'U'], [1, 'D'], [1, 'R'], [1, 'R'], [1, 'R']]


In [40]:
ids_time = last_time - first_time

In [41]:
ids_time

0.0921180248260498

In [42]:
len(all_expanded)

1806

## Test 2

In [43]:
first_time = time.time() 
answer = None
all_expanded = []
limit = 1

while answer == None:
    all_expanded.extend(expanded)
    expanded = []
    first_node = Node(maps[1], Cs[1], Ks[1], [], 0, [[0, 0, 0]], len(maps[1]), len(maps[1][0]))
    answer = first_node.RDLS(expanded, limit)
    limit += 1
last_time = time.time()

print("answer found")
print("path cost =", answer.path_cost)
print(answer.path)

answer found
path cost = 11
[[0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D'], [0, 'R'], [1, 'U'], [1, 'R'], [1, 'R'], [1, 'D'], [1, 'R']]


In [44]:
ids_time = last_time - first_time

In [45]:
ids_time

0.439774751663208

In [46]:
len(all_expanded)

1903

## Test 3

In [47]:
first_time = time.time() 
answer = None
all_expanded = []
limit = 19

while answer == None:
    all_expanded.extend(expanded)
    expanded = []
    first_node = Node(maps[2], Cs[2], Ks[2], [], 0, [[0, 0, 0]], len(maps[2]), len(maps[2][0]))
    answer = first_node.RDLS(expanded, limit)
    limit += 1
last_time = time.time()

print("answer found")
print("path cost =", answer.path_cost)
print(answer.path)

answer found
path cost = 19
[[0, 'D'], [0, 'D'], [0, 'R'], [0, 'R'], [0, 'U'], [0, 'R'], [0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D'], [0, 'D'], [1, 'R'], [1, 'R'], [1, 'R'], [1, 'U'], [1, 'D'], [1, 'R'], [1, 'R']]


In [48]:
ids_time = last_time - first_time

In [49]:
ids_time

8.211979389190674

In [50]:
len(all_expanded)

794

## Implementation with A* with consistent heuristic

### این الگوریتم کامل است و همواره جواب بهینه را می دهد و از نظر تعداد استیت ها و زمان اجرا نیز از دو الگوریتم قبلی بهتر است چون بر خلاف دو الگوریتم قبلی که جستجوی نا آگاهانه بودند این الگوریتم جستجوی آگاهانه است و همچنین ثابت می شود که بهینه ترین الگوریتم کاملی است که جواب بهینه را به دست می آورد 

In [51]:
maps_copy = deepcopy(maps)

## Implementation with weighted A* with Alfa = 1.3 

### این الگوریتم کامل .......... و همواره جواب بهینه را .............. ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

In [52]:
maps_copy = deepcopy(maps)

## Implementation with weighted A* with Alfa = 1.5

### این الگوریتم کامل .......... و همواره جواب بهینه را .............. ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

In [53]:
maps_copy = deepcopy(maps)